In [1]:
import optuna
from optuna import Trial

from math import sqrt
from typing import Tuple, List

import numpy as np
import pandas as pd
from mordred import Calculator, descriptors
#import openbabel
from openbabel import pybel
from PyBioMed.PyMolecule.fingerprint import CalculatePubChemFingerprint,CalculateECFP2Fingerprint
from rdkit import Chem
from rdkit.Chem.rdchem import Atom

from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score, confusion_matrix, matthews_corrcoef, roc_curve, auc 
from sklearn.metrics import precision_recall_curve

from sklearn.model_selection import StratifiedKFold, KFold


from torch_geometric.data import Data
from torch_geometric.loader import DataLoader as G_Loader 
from torch_geometric.nn import GCNConv, GATConv
from torch_geometric.nn import global_mean_pool
from torch_geometric.nn import BatchNorm


# RDkit
from rdkit import Chem
from rdkit.Chem.rdmolops import GetAdjacencyMatrix

from rdkit import Chem
from rdkit.Chem import MACCSkeys
from rdkit.Chem import rdMolDescriptors
from rdkit import DataStructs


# Pytorch and Pytorch Geometric
import torch

import torch.nn as nn
from torch.nn import Linear
import torch.optim as optim
import torch.nn.functional as F # activation function
from torch.utils.data import Dataset
from torch.utils.data import DataLoader as V_Loader # dataset management


%run ./graph_feature.ipynb 
%run ./dataset_processing.ipynb 

C:\Users\USER\anaconda3\envs\toxpred\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def compute_fingerprint_ECFP2(smiles_list: List[str]) -> np.ndarray:
    """
    Compute ECFP2 & PubChem fingerprint features for a list 
    of SMILES strings

    Parameters
    ----------
    smiles_list: List[str]
        The list of SMILES strings.

    Returns
    -------
    np.ndarray
        Returns a 2D numpy array, where each row corrsponds
        to the fingerprints of a SMILES strings in order.
    """
    molecular_mols = [Chem.MolFromSmiles(smi) for smi in smiles_list]
    # Initialize an array to store ECFP2 & PubChem fingerprint features
    features = np.zeros((len(smiles_list), 1024), dtype=np.int32)

    for i, mol in enumerate(molecular_mols):
        ECFP2_mol_fingerprint = CalculateECFP2Fingerprint(mol)
        numerical_representation = ECFP2_mol_fingerprint[0]
        features[i] = numerical_representation

    return features


def compute_fingerprint_PubChem(smiles_list: List[str]) -> np.ndarray:
    """
    Compute ECFP2 & PubChem fingerprint features for a list 
    of SMILES strings

    Parameters
    ----------
    smiles_list: List[str]
        The list of SMILES strings.

    Returns
    -------
    np.ndarray
        Returns a 2D numpy array, where each row corrsponds
        to the fingerprints of a SMILES strings in order.
    """
    molecular_mols = [Chem.MolFromSmiles(smi) for smi in smiles_list]
    # Initialize an array to store ECFP2 & PubChem fingerprint features
    features = np.zeros((len(smiles_list), 881), dtype=np.int32)

    for i, mol in enumerate(molecular_mols):
        pubchem_mol_fingerprint = CalculatePubChemFingerprint(mol)
        numerical_representation =  pubchem_mol_fingerprint
        features[i] = numerical_representation

    return features

def compute_fingerprint_MACCS(smiles_list: List[str]) -> np.ndarray:
    """
    Compute ECFP2 & PubChem fingerprint features for a list 
    of SMILES strings

    Parameters
    ----------
    smiles_list: List[str]
        The list of SMILES strings.

    Returns
    -------
    np.ndarray
        Returns a 2D numpy array, where each row corrsponds
        to the fingerprints of a SMILES strings in order.
    """
    molecular_mols = [Chem.MolFromSmiles(smi) for smi in smiles_list]
    # Initialize an array to store ECFP2 & PubChem fingerprint features
    features = np.zeros((len(smiles_list), 166), dtype=np.int32)

    for i, mol in enumerate(molecular_mols):
        one_fingerp =MACCSkeys.GenMACCSKeys(mol)
        list_one_fingerp = list(one_fingerp)
        del list_one_fingerp[0]
        numerical_representation = np.array(list_one_fingerp,dtype=np.int64)
        features[i] = numerical_representation

    return features

def compute_descriptor_features(smiles_list: List[str]) -> pd.DataFrame:
    """
    Compute 2D descriptor features for a list of SMILES strings

    Parameters
    ----------
    smiles_list: List[str]
        The list of SMILES strings.

    Returns
    -------
    np.ndarray
        Returns a pandas dataframe, where each row corrsponds
        to the descriptors of a SMILES strings in order.
    """
    descriptor_calc_2D = Calculator(descriptors, ignore_3D=True)
    molecular_mols = [Chem.MolFromSmiles(smi) for smi in smiles_list]
    descriptors_2D = descriptor_calc_2D.pandas(molecular_mols)
    return descriptors_2D


In [3]:
# GET ALL DATA 
pd_smiles,pd_labels = get_dataset('DILI-Dataset.csv')

# SPLIT DATA TRAIN AND TEST 80 AND 20 
X_train, X_test, y_train, y_test = train_test_split(pd_smiles, pd_labels, test_size=0.2, random_state=42)

In [4]:
pd_smiles_train = X_train
pd_smiles_test = X_test
pd_labels_train = y_train
pd_labels_test = y_test

In [5]:
list_y_smiles                   = list(pd_labels_train)
list_y_smiles_test              = list(pd_labels_test)
list_X_smiles                   = list(pd_smiles_train) 
list_X_smiles_test              = list(pd_smiles_test) 

In [6]:
# SHUFFLE FROM THE BEGINNInG 
# SHUFFLE TO SPREAD THE DATA WITH LABEL 0 AND 1 RANDOMLY AND CREATE K-FOLD CROSSVALIDATION
# ========================================================================================
k                               = 10
X_1,y_1                         = shuffle(list_X_smiles, list_y_smiles)
train_data                      = X_1
train_targets                   = y_1
all_train_indices, all_val_indices, total_train_data,total_train_targets,total_validation_data,total_validation_targets = CF_Validation_version_2(k,train_data,train_targets)

In [7]:
all_train_indices 

[1132, 1132, 1132, 1132, 1132, 1132, 1132, 1132, 1133, 1133]

In [8]:
# save all the index 
# train, validation, and test 
np.save('train_indices.npy', all_train_indices)
np.save('val_indices.npy', all_val_indices)
np.save('test_indices.npy', len(list_X_smiles_test))

In [9]:
# CONVERT TO VEC DATA FROM SMILES AFTER CROSS FOLD VALIDATION 
def convert2vec(input_data_smiles):
    fingerp_ECFP2 = compute_fingerprint_ECFP2(input_data_smiles)
    fingerp_PubChem = compute_fingerprint_PubChem(input_data_smiles)
    fingerp_MACCS = compute_fingerprint_MACCS(input_data_smiles)
    descriptors = compute_descriptor_features(input_data_smiles)
    
    return descriptors, fingerp_ECFP2,fingerp_PubChem,fingerp_MACCS

# convert the total train , validation, and test into vector data
def convert2vec_group(total_data):
    data_ECFP2 = [] 
    data_PubChem = []  
    data_MACCS = []  
    data_Desc = []  
    if len(total_data)> 2: # for data train and validation 
        for one_fold in total_data:
            collected_data = convert2vec(one_fold)
            data_Desc.append(collected_data[0])
            data_ECFP2.append(collected_data[1])
            data_PubChem.append(collected_data[2])
            data_MACCS.append(collected_data[3])
    else : # for data test 
        collected_data = convert2vec(total_data[0])
        data_Desc.append(collected_data[0])
        data_ECFP2.append(collected_data[1])
        data_PubChem.append(collected_data[2])
        data_MACCS.append(collected_data[3])
        
    return data_Desc,data_ECFP2,data_PubChem,data_MACCS

In [11]:
data_train = convert2vec_group(total_train_data)# fold x numberdata x number features(2 x 1) descriptor and fing           

data_train_Desc=  data_train[0]
data_train_ECFP2=  data_train[1]
data_train_PubChem=  data_train[2]
data_train_MACCS=  data_train[3]

  1%|▌                                                                                | 7/1132 [00:03<07:38,  2.45it/s]

C:\Users\USER\anaconda3\envs\toxpred\lib\site-packages\numpy\core\fromnumeric.py:86: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)


  6%|█████                                                                           | 72/1132 [00:07<01:15, 14.01it/s]

C:\Users\USER\anaconda3\envs\toxpred\lib\site-packages\numpy\core\fromnumeric.py:86: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)


  8%|██████▌                                                                         | 93/1132 [00:10<02:27,  7.02it/s]

...

 38%|█████████████████████████████▋                                                 | 426/1133 [01:01<01:29,  7.93it/s]

C:\Users\USER\anaconda3\envs\toxpred\lib\site-packages\numpy\core\fromnumeric.py:86: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)


 59%|██████████████████████████████████████████████▊                                | 672/1133 [01:39<01:03,  7.25it/s]

C:\Users\USER\anaconda3\envs\toxpred\lib\site-packages\numpy\core\fromnumeric.py:86: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)


100%|██████████████████████████████████████████████████████████████████████████████| 1133/1133 [02:28<00:00,  7.63it/s]


In [12]:
data_train_PubChem[0]

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [13]:
len(total_validation_data[0])

126

In [14]:
data_val = convert2vec_group(total_validation_data)# fold x numberdata x number features(2 x 1) descriptor and fing                
data_val_Desc=  data_val[0]
data_val_ECFP2=  data_val[1]
data_val_PubChem=  data_val[2]
data_val_MACCS=  data_val[3]

 21%|█████████████████▎                                                               | 27/126 [00:05<00:22,  4.39it/s]

C:\Users\USER\anaconda3\envs\toxpred\lib\site-packages\numpy\core\fromnumeric.py:86: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)
C:\Users\USER\anaconda3\envs\toxpred\lib\site-packages\numpy\core\fromnumeric.py:86: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)


 60%|████████████████████████████████████████████████▏                                | 75/126 [00:11<00:11,  4.47it/s]

C:\Users\USER\anaconda3\envs\toxpred\lib\site-packages\numpy\core\fromnumeric.py:86: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)
C:\Users\USER\anaconda3\envs\toxpred\lib\site-packages\numpy\core\fromnumeric.py:86: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)


 90%|████████████████████████████████████████████████████████████████████████▍       | 114/126 [00:15<00:02,  4.97it/s]

...

 38%|██████████████████████████████▍                                                  | 47/125 [00:07<00:06, 11.71it/s]

C:\Users\USER\anaconda3\envs\toxpred\lib\site-packages\numpy\core\fromnumeric.py:86: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)
C:\Users\USER\anaconda3\envs\toxpred\lib\site-packages\numpy\core\fromnumeric.py:86: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)
C:\Users\USER\anaconda3\envs\toxpred\lib\site-packages\numpy\core\fromnumeric.py:86: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)
C:\Users\USER\anaconda3\envs\toxpred\lib\site-packages\numpy\core\fromnumeric.py:86: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)


 74%|████████████████████████████████████████████████████████████▎                    | 93/125 [00:12<00:03,  9.76it/s]

C:\Users\USER\anaconda3\envs\toxpred\lib\site-packages\numpy\core\fromnumeric.py:86: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)


100%|████████████████████████████████████████████████████████████████████████████████| 125/125 [00:13<00:00,  9.51it/s]


In [15]:
len(list_X_smiles_test)
one_set_data_test = []
one_set_data_test.append(list_X_smiles_test)
len(one_set_data_test)

1

In [16]:

data_test = convert2vec_group(one_set_data_test)# fold x numberdata x number features(2 x 1) descriptor and fing                
data_test_Desc=  data_test[0]
data_test_ECFP2=  data_test[1]
data_test_PubChem=  data_test[2]
data_test_MACCS=  data_test[3]

 10%|████████▍                                                                        | 33/315 [00:04<01:35,  2.95it/s]

C:\Users\USER\anaconda3\envs\toxpred\lib\site-packages\numpy\core\fromnumeric.py:86: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)
C:\Users\USER\anaconda3\envs\toxpred\lib\site-packages\numpy\core\fromnumeric.py:86: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)
C:\Users\USER\anaconda3\envs\toxpred\lib\site-packages\numpy\core\fromnumeric.py:86: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)


 17%|█████████████▋                                                                   | 53/315 [00:07<00:33,  7.76it/s]

C:\Users\USER\anaconda3\envs\toxpred\lib\site-packages\numpy\core\fromnumeric.py:86: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)


 31%|█████████████████████████▏                                                       | 98/315 [00:11<00:20, 10.68it/s]

...

 65%|███████████████████████████████████████████████████▊                            | 204/315 [00:36<00:28,  3.83it/s]

C:\Users\USER\anaconda3\envs\toxpred\lib\site-packages\mordred\_matrix_attributes.py:251: RuntimeWarning: invalid value encountered in double_scalars
  s += (eig.vec[i, eig.max] * eig.vec[j, eig.max]) ** -0.5


 69%|██████████████████████████████████████████████████████▊                         | 216/315 [00:36<00:18,  5.41it/s]

C:\Users\USER\anaconda3\envs\toxpred\lib\site-packages\numpy\core\fromnumeric.py:86: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)


100%|████████████████████████████████████████████████████████████████████████████████| 315/315 [00:49<00:00,  6.33it/s]


# 1. Data Repairing 
## finding the not number value and replace it with zero value 
## finding also Nan number value and replace it also with zero value

In [17]:
import numbers
import math 
def coerce_to_numeric(value):
    if isinstance(value, numbers.Number) and math.isnan(value)==False:
        return value
    else:
        return 0

# Returns a cleaned version of df[col1]
#clean_col = df[col1].apply(coerce_to_numeric)
#for data in descriptors_train.loc[0]:
#    print(coerce_to_numeric(data))

# input is list of descriptors and change it into data list
def find_notnumber_n_replace(descriptors):
    list_data= []
    for j in range(len(descriptors)):
        one_feature=[]
        for data in descriptors.loc[j]:
            clean_data = coerce_to_numeric(data)
            one_feature.append(clean_data)
        list_data.append(one_feature)
    return list_data

In [18]:
# REPAIR ALL DATA
# data repairing only used for descriptor type data
def repair_data(total_vec_data):
    total_data=[]
    if len(total_vec_data)>2 :
        for data_fold in total_vec_data:
            list_data = find_notnumber_n_replace(data_fold)
            total_data.append(list_data)
    else :

        list_data = find_notnumber_n_replace(total_vec_data[0])
        total_data.append(list_data)
    return total_data

#descriptors = data_vec_train[0][0] # descriptor in fold 0
#list_data = find_notnumber_n_replace(descriptors)

In [19]:
len(data_test_Desc)

1

In [20]:
data_train_Desc1 = repair_data(data_train_Desc)
data_val_Desc1 = repair_data(data_val_Desc)
data_test_Desc1 = repair_data(data_test_Desc)

# 3. Standarization (Max-Min Scaler)
only DESCRIPTOR that would be standardize 


the FINGERPRINTS have binary value 0 and 1 , dont need to be scaled

In [21]:
from sklearn.preprocessing import StandardScaler, Normalizer, MinMaxScaler
from sklearn.model_selection import train_test_split

# scale only descriptor 
# chi square test need positive data 

In [22]:
#Normalizer = Normalizer()
Scaler = MinMaxScaler()

In [23]:
# join only the descriptor 
filtered_desc_train = np.concatenate((data_train_Desc1[0],data_val_Desc1[0]),axis =0)

In [24]:
scaler1       = Scaler.fit(filtered_desc_train)

scaled_desc_train=[]
scaled_desc_val=[]
for data_train,data_val in zip (data_train_Desc1, data_val_Desc1): 
    scaled_desc_train.append(scaler1.transform(data_train))
    scaled_desc_val.append(scaler1.transform(data_val))
    
scaled_desc_test = scaler1.transform(data_test_Desc1[0])

In [25]:
len(scaled_desc_test)

315

# 5. Create Graph Data 

In [26]:
# RDkit
from rdkit import Chem
from rdkit.Chem.rdmolops import GetAdjacencyMatrix
# Pytorch and Pytorch Geometric
import torch

import torch.nn as nn
from torch.nn import Linear
import torch.optim as optim
import torch.nn.functional as F # activation function
from torch.utils.data import Dataset, DataLoader # dataset management

In [27]:
# CONVERT THE DATASET INTO GRAPH STRUCTURED DATA 
# BOTH DATA_TRAIN AND DATA_VALIDATION ARE PREPARED IN GRAPH STRUCTURED DATA 
#=============================================================================================
data_list_train = []
data_list_val =[]
for X_train,y_train,X_val,y_val in zip(total_train_data, total_train_targets, total_validation_data, total_validation_targets):
    data_graph_train = create_pytorch_geometric_graph_data_list_from_smiles_and_labels(X_train, y_train)
    data_graph_val   = create_pytorch_geometric_graph_data_list_from_smiles_and_labels(X_val, y_val)
    
    data_list_train.append(data_graph_train)
    data_list_val.append(data_graph_val)

    
# INDEPENDENT DATASET IN GRAPH STRUCTURED DATA
#===========================================================================================
x_smiles = list_X_smiles_test
y = list_y_smiles_test

data_list_test = create_pytorch_geometric_graph_data_list_from_smiles_and_labels(x_smiles, y)

# 6. Save the preprocessed data 

In [28]:
# save descriptors 
# save in numpy format 
np.save('final_clean_desc_train0.npy',scaled_desc_train[0])
np.save('final_clean_desc_train1.npy',scaled_desc_train[1])
np.save('final_clean_desc_train2.npy',scaled_desc_train[2])
np.save('final_clean_desc_train3.npy',scaled_desc_train[3])
np.save('final_clean_desc_train4.npy',scaled_desc_train[4])
np.save('final_clean_desc_train5.npy',scaled_desc_train[5])
np.save('final_clean_desc_train6.npy',scaled_desc_train[6])
np.save('final_clean_desc_train7.npy',scaled_desc_train[7])
np.save('final_clean_desc_train8.npy',scaled_desc_train[8])
np.save('final_clean_desc_train9.npy',scaled_desc_train[9])
# save the data vector 
# this is saved in numpy data 
np.save('final_clean_desc_val0.npy',scaled_desc_val[0])
np.save('final_clean_desc_val1.npy',scaled_desc_val[1])
np.save('final_clean_desc_val2.npy',scaled_desc_val[2])
np.save('final_clean_desc_val3.npy',scaled_desc_val[3])
np.save('final_clean_desc_val4.npy',scaled_desc_val[4])
np.save('final_clean_desc_val5.npy',scaled_desc_val[5])
np.save('final_clean_desc_val6.npy',scaled_desc_val[6])
np.save('final_clean_desc_val7.npy',scaled_desc_val[7])
np.save('final_clean_desc_val8.npy',scaled_desc_val[8])
np.save('final_clean_desc_val9.npy',scaled_desc_val[9])

np.save('final_clean_desc_test.npy',scaled_desc_test)

In [29]:
# data for fingerprint_ECFP2
np.save('final_clean_fingerp_ECFP2_train0.npy',data_train_ECFP2[0])
np.save('final_clean_fingerp_ECFP2_train1.npy',data_train_ECFP2[1])
np.save('final_clean_fingerp_ECFP2_train2.npy',data_train_ECFP2[2])
np.save('final_clean_fingerp_ECFP2_train3.npy',data_train_ECFP2[3])
np.save('final_clean_fingerp_ECFP2_train4.npy',data_train_ECFP2[4])
np.save('final_clean_fingerp_ECFP2_train5.npy',data_train_ECFP2[5])
np.save('final_clean_fingerp_ECFP2_train6.npy',data_train_ECFP2[6])
np.save('final_clean_fingerp_ECFP2_train7.npy',data_train_ECFP2[7])
np.save('final_clean_fingerp_ECFP2_train8.npy',data_train_ECFP2[8])
np.save('final_clean_fingerp_ECFP2_train9.npy',data_train_ECFP2[9])

# save the data vector 
# this is saved in numpy data 
np.save('final_clean_fingerp_ECFP2_val0.npy',data_val_ECFP2[0])
np.save('final_clean_fingerp_ECFP2_val1.npy',data_val_ECFP2[1])
np.save('final_clean_fingerp_ECFP2_val2.npy',data_val_ECFP2[2])
np.save('final_clean_fingerp_ECFP2_val3.npy',data_val_ECFP2[3])
np.save('final_clean_fingerp_ECFP2_val4.npy',data_val_ECFP2[4])
np.save('final_clean_fingerp_ECFP2_val5.npy',data_val_ECFP2[5])
np.save('final_clean_fingerp_ECFP2_val6.npy',data_val_ECFP2[6])
np.save('final_clean_fingerp_ECFP2_val7.npy',data_val_ECFP2[7])
np.save('final_clean_fingerp_ECFP2_val8.npy',data_val_ECFP2[8])
np.save('final_clean_fingerp_ECFP2_val9.npy',data_val_ECFP2[9])

np.save('final_clean_fingerp_ECFP2_test.npy',data_test_ECFP2)

In [31]:
# data for fingerprint_PubChem
np.save('final_clean_fingerp_PubChem_train0.npy',data_train_PubChem[0])
np.save('final_clean_fingerp_PubChem_train1.npy',data_train_PubChem[1])
np.save('final_clean_fingerp_PubChem_train2.npy',data_train_PubChem[2])
np.save('final_clean_fingerp_PubChem_train3.npy',data_train_PubChem[3])
np.save('final_clean_fingerp_PubChem_train4.npy',data_train_PubChem[4])
np.save('final_clean_fingerp_PubChem_train5.npy',data_train_PubChem[5])
np.save('final_clean_fingerp_PubChem_train6.npy',data_train_PubChem[6])
np.save('final_clean_fingerp_PubChem_train7.npy',data_train_PubChem[7])
np.save('final_clean_fingerp_PubChem_train8.npy',data_train_PubChem[8])
np.save('final_clean_fingerp_PubChem_train9.npy',data_train_PubChem[9])

# save the data vector 
# this is saved in numpy data 
np.save('final_clean_fingerp_PubChem_val0.npy',data_val_PubChem[0])
np.save('final_clean_fingerp_PubChem_val1.npy',data_val_PubChem[1])
np.save('final_clean_fingerp_PubChem_val2.npy',data_val_PubChem[2])
np.save('final_clean_fingerp_PubChem_val3.npy',data_val_PubChem[3])
np.save('final_clean_fingerp_PubChem_val4.npy',data_val_PubChem[4])
np.save('final_clean_fingerp_PubChem_val5.npy',data_val_PubChem[5])
np.save('final_clean_fingerp_PubChem_val6.npy',data_val_PubChem[6])
np.save('final_clean_fingerp_PubChem_val7.npy',data_val_PubChem[7])
np.save('final_clean_fingerp_PubChem_val8.npy',data_val_PubChem[8])
np.save('final_clean_fingerp_PubChem_val9.npy',data_val_PubChem[9])

np.save('final_clean_fingerp_PubChem_test.npy',data_test_PubChem)

In [32]:
# data for fingerprint_MACCS
np.save('final_clean_fingerp_MACCS_train0.npy',data_train_MACCS[0])
np.save('final_clean_fingerp_MACCS_train1.npy',data_train_MACCS[1])
np.save('final_clean_fingerp_MACCS_train2.npy',data_train_MACCS[2])
np.save('final_clean_fingerp_MACCS_train3.npy',data_train_MACCS[3])
np.save('final_clean_fingerp_MACCS_train4.npy',data_train_MACCS[4])
np.save('final_clean_fingerp_MACCS_train5.npy',data_train_MACCS[5])
np.save('final_clean_fingerp_MACCS_train6.npy',data_train_MACCS[6])
np.save('final_clean_fingerp_MACCS_train7.npy',data_train_MACCS[7])
np.save('final_clean_fingerp_MACCS_train8.npy',data_train_MACCS[8])
np.save('final_clean_fingerp_MACCS_train9.npy',data_train_MACCS[9])

# save the data vector 
# this is saved in numpy data 
np.save('final_clean_fingerp_MACCS_val0.npy',data_val_MACCS[0])
np.save('final_clean_fingerp_MACCS_val1.npy',data_val_MACCS[1])
np.save('final_clean_fingerp_MACCS_val2.npy',data_val_MACCS[2])
np.save('final_clean_fingerp_MACCS_val3.npy',data_val_MACCS[3])
np.save('final_clean_fingerp_MACCS_val4.npy',data_val_MACCS[4])
np.save('final_clean_fingerp_MACCS_val5.npy',data_val_MACCS[5])
np.save('final_clean_fingerp_MACCS_val6.npy',data_val_MACCS[6])
np.save('final_clean_fingerp_MACCS_val7.npy',data_val_MACCS[7])
np.save('final_clean_fingerp_MACCS_val8.npy',data_val_MACCS[8])
np.save('final_clean_fingerp_MACCS_val9.npy',data_val_MACCS[9])

np.save('final_clean_fingerp_MACCS_test.npy',data_test_MACCS)

In [33]:
import os
# save data train
for i in range(10):
    path = './data_train_' + str(i)
    # create new single directory
    os.mkdir(path)
    for idx, tensor in enumerate(data_list_train[i]):
        torch.save(tensor, f"data_train_{i}/tensor{idx}.pt")

# save data validation        
for i in range(10):
    path = './data_val_' + str(i)
    # create new single directory
    os.mkdir(path)
    for idx, tensor in enumerate(data_list_val[i]):
        torch.save(tensor, f"data_val_{i}/tensor{idx}.pt")
        
# save data test 
path = './data_test' 
# create new single directory
os.mkdir(path)
for idx, tensor in enumerate(data_list_test):
    torch.save(tensor, f"data_test/tensor{idx}.pt")

In [34]:
# save the target 
# save the label also 
# save the data vector (only use variance threshold)
# this is saved in numpy data 
np.save('total_train_targets0.npy',total_train_targets[0])
np.save('total_train_targets1.npy',total_train_targets[1])
np.save('total_train_targets2.npy',total_train_targets[2])
np.save('total_train_targets3.npy',total_train_targets[3])
np.save('total_train_targets4.npy',total_train_targets[4])
np.save('total_train_targets5.npy',total_train_targets[5])
np.save('total_train_targets6.npy',total_train_targets[6])
np.save('total_train_targets7.npy',total_train_targets[7])
np.save('total_train_targets8.npy',total_train_targets[8])
np.save('total_train_targets9.npy',total_train_targets[9])

# save the data vector 
# this is saved in numpy data 
np.save('total_validation_targets0.npy',total_validation_targets[0])
np.save('total_validation_targets1.npy',total_validation_targets[1])
np.save('total_validation_targets2.npy',total_validation_targets[2])
np.save('total_validation_targets3.npy',total_validation_targets[3])
np.save('total_validation_targets4.npy',total_validation_targets[4])
np.save('total_validation_targets5.npy',total_validation_targets[5])
np.save('total_validation_targets6.npy',total_validation_targets[6])
np.save('total_validation_targets7.npy',total_validation_targets[7])
np.save('total_validation_targets8.npy',total_validation_targets[8])
np.save('total_validation_targets9.npy',total_validation_targets[9])


total_test_targets = np.array(list_y_smiles_test)
np.save('total_test_targets.npy',total_test_targets)